# Implementação Parcial – Etapa 3
## Previsão e Monitoramento da Epidemia de Dengue: Uma Abordagem por Séries Temporais
**Jéssica Clara da Silva Santos**  
Faculdade de Computação e Informática (FCI), Universidade Presbiteriana Mackenzie – São Paulo, SP – Brasil  
10414974@mackenzista.com.br


## 1. Instalação de Dependências (Colab ou local)

In [ ]:
# Se estiver no Google Colab, execute para instalar pacotes
# !pip install pandas numpy matplotlib statsmodels scikit-learn


## 2. Imports e Configurações Iniciais

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_rows', 10)
plt.style.use('seaborn')


## 3. Coleta de Dados via CSV
Carregamento de dados históricos de dengue para o município de São Paulo.
- **Local**: `dados/dengue_sp.csv` no repositório
- **Raw GitHub**: forneça a URL crua do arquivo CSV

In [ ]:
# Exemplo local
# data = pd.read_csv('dados/dengue_sp.csv', parse_dates=['date'], index_col='date')

# Exemplo via GitHub raw
url_csv = 'https://raw.githubusercontent.com/jehclara14/Projeto-Aplicado-IV/main/dados/dengue_sp.csv'
data = pd.read_csv(url_csv, parse_dates=['date'], index_col='date')

print(f"Período carregado: {data.index.min().date()} até {data.index.max().date()}")
print(f"Total de observações: {len(data)}")


## 4. Pré‑processamento de Dados

In [ ]:
# Imputação de valores ausentes
data['cases'] = data['cases'].interpolate(method='linear')

# Remoção de outliers via IQR
Q1 = data['cases'].quantile(0.25)
Q3 = data['cases'].quantile(0.75)
IQR = Q3 - Q1
mask = (data['cases'] >= Q1 - 1.5 * IQR) & (data['cases'] <= Q3 + 1.5 * IQR)
data = data.loc[mask]

data.head()


## 5. Visualização e Análise Exploratória

In [ ]:
# Série temporal de casos
plt.figure(figsize=(12, 4))
plt.plot(data['cases'], label='Casos de dengue')
plt.title('Casos Semanais de Dengue (SP)')
plt.xlabel('Data')
plt.ylabel('Casos')
plt.legend()
plt.show()

from statsmodels.tsa.seasonal import seasonal_decompose
decomp = seasonal_decompose(data['cases'], period=52, model='additive', extrapolate_trend='freq')
fig = decomp.plot()
fig.set_size_inches(12, 8)
plt.show()


## 6. Modelagem Estatística (SARIMA)

In [ ]:
sarima_model = sm.tsa.statespace.SARIMAX(
    data['cases'],
    order=(1,1,1),
    seasonal_order=(1,1,1,52),
    enforce_stationarity=False,
    enforce_invertibility=False
)
sarima_results = sarima_model.fit(disp=False)
print(sarima_results.summary().tables[0])


## 7. Modelagem com Random Forest

In [ ]:
# Preparação de features com janela
n_lags = 4  # 4 semanas de lag
X, y = [], []
values = data['cases'].values
for i in range(n_lags, len(values)):
    X.append(values[i-n_lags:i])
    y.append(values[i])
X, y = np.array(X), np.array(y)

# Divisão treino/teste
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)


## 8. Avaliação de Desempenho

In [ ]:
# SARIMA forecast
sarima_pred = sarima_results.predict(start=0, end=len(data)-1)
mae_sarima = mean_absolute_error(data['cases'], sarima_pred)
rmse_sarima = np.sqrt(mean_squared_error(data['cases'], sarima_pred))

# Random Forest forecast
mae_rf = mean_absolute_error(y_test, rf_pred)
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_pred))

print(f"SARIMA MAE: {mae_sarima:.2f}, RMSE: {rmse_sarima:.2f}")
print(f"RF     MAE: {mae_rf:.2f},   RMSE: {rmse_rf:.2f}")


## 9. Próximos Passos
- Ajustar hiperparâmetros via grid search
- Explorar modelos bayesianos
- Desenvolver dashboards interativos
